<a href="https://colab.research.google.com/github/achraf-halla/ADME/blob/main/Solubility_AqSolDB_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil

shutil.rmtree('/content/grover')

In [2]:
%cd drive/MyDrive
!git clone https://github.com/atreyamaj/grover.git

[Errno 2] No such file or directory: 'drive/MyDrive'
/content
Cloning into 'grover'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 206 (delta 11), reused 4 (delta 4), pack-reused 181 (from 1)
Receiving objects: 100% (206/206), 2.22 MiB | 8.06 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [3]:
%cd grover

/content/grover


In [4]:
!pip install -e ./

Obtaining file:///content/grover
  Preparing metadata (setup.py) ... done
  Running setup.py develop for grover


In [5]:
!pip install deepchem
!pip install git+https://github.com/bp-kelley/descriptastorus

  Cloning https://github.com/bp-kelley/descriptastorus to /tmp/pip-req-build-lkilldmm
  Running command git clone --filter=blob:none --quiet https://github.com/bp-kelley/descriptastorus /tmp/pip-req-build-lkilldmm
  Resolved https://github.com/bp-kelley/descriptastorus to commit 9a190343bcd3cfd35142d378d952613bcac40797
  Preparing metadata (setup.py) ... done


In [6]:
import pandas as pd
df = pd.read_csv("exampledata/finetune/solubility_aqsoldb.tab", sep='\t')

In [7]:
df = df.iloc[:,1:]

In [8]:
df.head()

,Drug,Y
0,CCCCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],-3.616127
1,O=C1Nc2cccc3cccc1c23,-3.254767
2,O=Cc1ccc(Cl)cc1,-2.177078
3,CC(c1ccccc1)c1cc(C(=O)[O-])c(O)c(C(C)c2ccccc2)...,-3.924409
4,c1cc(N(CC2CO2)CC2CO2)ccc1Cc1ccc(N(CC2CO2)CC2CO...,-4.662065


In [9]:
df.to_csv("exampledata/finetune/solubility.csv")

In [10]:
import os
os. getcwd()

'/content/grover'

In [11]:
!python scripts/save_features.py --data_path exampledata/finetune/solubility.csv \
                                --save_path exampledata/finetune/solubility.npz \
                                --features_generator rdkit_2d_normalized \
                                --restart

Traceback (most recent call last):
  File "/content/grover/scripts/save_features.py", line 127, in <module>
    generate_and_save_features(args)
  File "/content/grover/scripts/save_features.py", line 54, in generate_and_save_features
    data = get_data(path=args.data_path, max_data_size=None)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/grover/grover/util/utils.py", line 203, in get_data
    data = MoleculeDataset([
                           ^
  File "/content/grover/grover/util/utils.py", line 204, in <listcomp>
    MoleculeDatapoint(
  File "/content/grover/grover/data/moldataset.py", line 75, in __init__
    self.targets = [float(x) if x != '' else None for x in line[1:]]
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/grover/grover/data/moldataset.py", line 75, in <listcomp>
    self.targets = [float(x) if x != '' else None for x in line[1:]]
                    ^^^^^^^^
ValueError: could not convert string t

In [13]:
df1 = pd.read_csv("exampledata/finetune/bbbp.csv")

In [14]:
df1.head()

,smiles,p_np
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,1
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,1
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,1
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,1
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,1


In [15]:
!python main.py finetune --data_path exampledata/finetune/solubility.csv \
                        --features_path exampledata/finetune/solubility.npz \
                        --save_dir model/finetune/solubility/ \
                        --checkpoint_path model/tryout/model.ep3 \
                        --dataset_type regression \
                        --split_type scaffold_balanced \
                        --ensemble_size 1 \
                        --num_folds 3 \
                        --no_features_scaling \
                        --ffn_hidden_size 200 \
                        --batch_size 32 \
                        --epochs 10 \
                        --init_lr 0.00015

[WARNING] Horovod cannot be imported; multi-GPU training is unsupported
Fold 0
Traceback (most recent call last):
  File "/content/grover/main.py", line 40, in <module>
    cross_validate(args, logger)
  File "/content/grover/task/cross_validate.py", line 42, in cross_validate
    model_scores = run_training(args, time_start, logger)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/grover/task/train.py", line 122, in run_training
    torch.cuda.set_device(idx)
  File "/usr/local/lib/python3.11/dist-packages/torch/cuda/__init__.py", line 476, in set_device
    torch._C._cuda_setDevice(device)
  File "/usr/local/lib/python3.11/dist-packages/torch/cuda/__init__.py", line 319, in _lazy_init
    torch._C._cuda_init()
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx
